# End to End Chicken Disease Classification

## Stage 2 : Prepare Base Model

In [1]:
%pwd

'D:\\Documents\\Projects\\chiicken_disease_classification\\research'

In [2]:
import os
os.chdir("../")

In [3]:
%pwd

'D:\\Documents\\Projects\\chiicken_disease_classification'

### Update entity

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_model_path: Path
    params_learning_rate: float
    params_image_size: list
    params_include_top: bool
    params_weights: str
    params_classes: int

### Update configuration manager in src config

In [5]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILEPATH,
                 params_filepath = PARAMS_FILEPATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        create_directories([config.root_dir])
        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir = Path(config.root_dir),
            base_model_path = Path(config.base_model_path),
            updated_model_path = Path(config.updated_model_path),
            params_learning_rate= self.params.LEARNING_RATE,
            params_image_size = self.params.IMAGE_SIZE,
            params_classes= self.params.CLASSES,
            params_weights = self.params.WEIGHTS,
            params_include_top=self.params.INCLUDE_TOP
        )
        
        
        return prepare_base_model_config

### Update the components

In [7]:
import tensorflow as tf

In [10]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size,
            weights=self.config.params_weights,
            include_top = self.config.params_include_top
        )
        self.save_model(
            path = self.config.base_model_path,
            model = self.model
        )
    
    @staticmethod
    def prepare_full_model(model: tf.keras.Model,
                           classes: int,
                           freeze_all: bool,
                           freeze_till: None,
                           learning_rate:float) -> tf.keras.Model:
        if freeze_all:
            for layer in model.layers:
                layer.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False
                
        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(classes, activation="softmax")(flatten_in)
                
        full_model = tf.keras.models.Model(
            inputs = model.input,
            outputs = prediction
        )
        
        full_model.compile(
            loss = "sparse_categorical_crossentropy,
            optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate),
            metrics = ["accuracy"]
        )
        
        full_model.summary()
        return full_model       
    
    def update_base_model(self):
        self.full_model = self.prepare_full_model(
            model = self.model,
            classes = self.config.params_classes,
            freeze_all = True,
            freeze_till = None,
            learning_rate = self.config.params_learning_rate            
        )
        
        self.save_model(
            path = self.config.updated_model_path,
            model = self.full_model
        )
    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

### Update pipeline

In [11]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2023-10-18 23:59:10,763:  INFO: common: yaml file: config\config.yaml loaded]
[2023-10-18 23:59:10,772:  INFO: common: yaml file: params.yaml loaded]
[2023-10-18 23:59:10,775:  INFO: common: Created directory at artifacts]
[2023-10-18 23:59:10,778:  INFO: common: Created directory at artifacts/prepare_base_model]
[2023-10-18 23:59:11,653:  WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]


TypeError: ('Keyword argument not understood:', 'output')